In [ ]:
NGROK_URL='http://09c34350b9e4.ngrok.io'
EXPERIMENT='pelouches'
EPOCHS=50
VERSION=1

SAVE_PATH='saved_model'
%mkdir saved_model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%writefile -a requirements.txt
tensorflow==2.1.0
tensorflow-hub==0.8.0
matplotlib==3.3.1
numpy==1.19.1
mlflow==1.11.0
seaborn==0.11.0

Writing requirements.txt


In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 102kB 11.6MB/s 
     |████████████████████████████████| 11.6MB 25.2MB/s 
     |████████████████████████████████| 14.5MB 316kB/s 
     |████████████████████████████████| 13.9MB 322kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 450kB 36.8MB/s 
     |████████████████████████████████| 3.9MB 46.4MB/s 
     |████████████████████████████████| 1.1MB 42.6MB/s 
     |████████████████████████████████| 153kB 47.3MB/s 
     |████████████████████████████████| 163kB 46.4MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 327kB 44.8MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 6.0MB 44.1MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 204kB 50.3MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Sequential
from tensorflow.compat.v1.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import mlflow 
import argparse
import os 
import time

In [ ]:
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [ ]:
train_root = "/content/gdrive/My Drive/Colab Notebooks/datasets/pelouches/train"
test_root = "/content/gdrive/My Drive/Colab Notebooks/datasets/pelouches/test"

In [ ]:
#create a custom callback to visualize the training progress during every epoch.
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []
    
  def on_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['accuracy'])

In [ ]:
#Train code
train_generator = ImageDataGenerator(rescale=1/255) 
test_generator = ImageDataGenerator(rescale=1/255) 

train_image_data = train_generator.flow_from_directory(str(train_root),target_size=(224,224))
test_image_data = test_generator.flow_from_directory(str(test_root), target_size=(224,224))

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
feature_extractor_url = r"https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2"

In [ ]:
IMAGE_SIZE = hub.get_expected_image_size(hub.Module(feature_extractor_url))
IMAGE_SIZE

[224, 224]

In [ ]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                   input_shape=IMAGE_SIZE+[3], 
                                   trainable=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
model = Sequential([
        feature_extractor_layer,
        layers.Dense(train_image_data.num_classes, activation = "softmax")
        ])

In [ ]:
#Mlflow settings
#set MLflow server 
mlflow.set_tracking_uri(NGROK_URL)
#Set experiment
if mlflow.get_experiment_by_name(EXPERIMENT) != None:
    exp_id = mlflow.set_experiment(EXPERIMENT)
else: 
    exp_id = mlflow.create_experiment(EXPERIMENT)

#Close active runs
if mlflow.active_run():
    mlflow.end_run()

In [ ]:
# initialize the TFHub module
sess = K.get_session() 
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
loss='categorical_crossentropy'

model.compile(
        optimizer = tf.train.AdamOptimizer(),
        loss = loss,
        metrics = ['accuracy']
        )

In [ ]:
# Early stopping to stop the training if loss start to increase. It also avoids overvitting.
es = EarlyStopping(patience=2,monitor="val_loss")

In [ ]:
#use CallBacks to record accuracy and loss.
batch_stats = CollectBatchStats()

In [ ]:
def image_load(image_path):
    loaded_image = image.load_img(image_path)
    image_rel = pathlib.Path(image_path).relative_to(train_root)
    print(image_rel)
    return loaded_image

def feature_extractor(x):
  feature_extractor_module = hub.Module(feature_extractor_url)
  return feature_extractor_module(x)

def evaluate_model(model, x_test, y_test):
    """
    Evaluate the model with unseen and untrained data
    :param model:
    :return: results of probability
    """

    return model.evaluate(x_test, y_test)

def get_binary_loss(hist):
    loss = hist.history['loss']
    loss_val = loss[len(loss) - 1]
    return loss_val

def get_binary_acc(hist):
    acc = hist.history['accuracy']
    acc_value = acc[len(acc) - 1]

    return acc_value

def get_validation_loss(hist):
    val_loss = hist.history['val_loss']
    val_loss_value = val_loss[len(val_loss) - 1]

    return val_loss_value

def get_validation_acc(hist):
    val_acc = hist.history['val_accuracy']
    val_acc_value = val_acc[len(val_acc) - 1]

    return val_acc_value


def print_metrics(hist):

    acc_value = get_binary_acc(hist)
    loss_value = get_binary_loss(hist)

    val_acc_value = get_validation_acc(hist)

    val_loss_value = get_validation_loss(hist)

    print("Final metrics: binary_loss:%6.4f" % loss_value)
    print("Final metrics: binary_accuracy=%6.4f" % acc_value)
    print("Final metrics: validation_binary_loss:%6.4f" % val_loss_value)
    print("Final metrics: validation_binary_accuracy:%6.4f" % val_acc_value)

In [ ]:
t = time.time()
SAVE_PATH=os.path.join(SAVE_PATH, "{}".format(int(t)))

with mlflow.start_run(run_id=None, experiment_id=exp_id, run_name=None, nested=False): 

      # fitting the model
      history = model.fit((item for item in train_image_data), epochs=EPOCHS,
              steps_per_epoch=21,
              callbacks = [batch_stats, es],validation_data=test_image_data)

      #mlflow autolog
      #mlflow.tensorflow.autolog()

      #Set tags
      tags={}
      tags['name']=EXPERIMENT
      tags['version']=VERSION
      mlflow.set_tags(tags)

      #mlflow logging
      # log parameters
      #mlflow.log_param("hidden_layers", args.hidden_layers)
      #mlflow.log_param("output", args.output)
      mlflow.log_param("epochs", EPOCHS)
      mlflow.log_param("loss_function", loss)
      # log metrics
      mlflow.log_metric("binary_loss", get_binary_loss(history))
      mlflow.log_metric("binary_acc",  get_binary_acc(history))
      mlflow.log_metric("validation_loss", get_binary_loss(history))
      mlflow.log_metric("validation_acc", get_validation_acc(history))
      #results=evaluate_model() #TODO
      #mlflow.log_metric("average_loss", results[0])
      #mlflow.log_metric("average_acc", results[1])
      
      #log model
      #model.save(os.path.join(BASE_DIR, "models", "{}.h5".format(int(t)))) #HDF5 format
      tf.saved_model.save(model, SAVE_PATH) #SavedModel format
      #mlflow.tensorflow.log_model(model, 'model') #TODO fix
      
      # log artifacts (matplotlib images for loss/accuracy)
      #mlflow.log_artifacts(image_dir)

      #print labels
      label_names = sorted(train_image_data.class_indices.items(), key=lambda pair:pair[1])
      label_names = np.array([key.title() for key, value in label_names])
      print(label_names)

      mlflow.end_run()

Epoch 1/50
21/21 [==============================] - 58s 3s/step - loss: 0.1999 - accuracy: 0.9514 - val_loss: 0.0791 - val_accuracy: 1.0000
Epoch 2/50
21/21 [==============================] - 18s 862ms/step - loss: 0.0217 - accuracy: 1.0000 - val_loss: 0.0564 - val_accuracy: 1.0000
Epoch 3/50
21/21 [==============================] - 19s 902ms/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 0.0361 - val_accuracy: 1.0000
Epoch 4/50
21/21 [==============================] - 18s 870ms/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.0296 - val_accuracy: 1.0000
Epoch 5/50
21/21 [==============================] - 19s 904ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0260 - val_accuracy: 1.0000
Epoch 6/50
21/21 [==============================] - 18s 862ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0231 - val_accuracy: 1.0000
Epoch 7/50
21/21 [==============================] - 19s 907ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0211 - val_accuracy: 1.0000
Ep

INFO:tensorflow:Assets written to: saved_model/1602777973/assets


INFO:tensorflow:Assets written to: saved_model/1602777973/assets


['Kitten' 'Rufy']


In [ ]:
%%bash -s $SAVE_PATH
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


2020-10-15 16:22:36.934041: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-10-15 16:22:36.934166: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-10-15 16:22:36.934180: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [ ]:
import matplotlib.image as mpimg 
from matplotlib.pyplot import imshow
%matplotlib inline

def show_img(image):
  testim = mpimg.imread(image)
  imshow(testim)

In [ ]:
#input data
import cv2

dim=tuple([224, 224])
img_test='dataset/test/Orange/Orange_054.jpg'
#show_img(img_test)

#resize image
img_test=cv2.imread(img_test)
img_test=cv2.resize(img_test, dim, interpolation=cv2.INTER_AREA)
print(img_test.shape)
img_test = np.array(img_test).reshape(-1, 224, 224, 3)
img_test.shape

In [ ]:
#load model, see: https://medium.com/@jsflo.dev/saving-and-loading-a-tensorflow-model-using-the-savedmodel-api-17645576527
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ["serve"], SAVE_PATH)
    graph = tf.get_default_graph()
    print(graph.get_operations())

    input_tensor = graph.get_tensor_by_name("serving_default_keras_layer_input:0")
    output_tensor = graph.get_tensor_by_name("StatefulPartitionedCall:0")
    print(input_tensor)
    print(output_tensor)

    feed_dict ={input_tensor:img_test}

    #test model on an image
    prediction=sess.run(output_tensor,feed_dict)[0]
    index=np.argmax(prediction, axis=0)
    print(prediction)
    print(label_names[index])